In [1]:
import os
import pandas as pd
import numpy as np
import csv
from itertools import islice
from scipy import stats

In [2]:
def getIPDict(ip_file):
	"""
    Using code provided by Sudalai :)
    
	Each line in the IP file belongs to either a device or a cookie.
	This function creates two dictionaries
	1. Device dictionary - has device_id as key and the IPs it belongs to as value in the form of list
	2. IP Dictionary - has IP address as the key and the cookies in that IP as value in the form of list
	Reasoning:
	We need to find the cookies that are associated with the given device in the competition. So given a device we can find out the IP addressess of the device from device dictionary. Then using those IP addresses and IP Dictionary, find out the cookies associated with the IP and link them back to the device.
	"""
	# reading the ip file #
	reader = csv.reader(ip_file)
	header = reader.next()             # skipping the header

	# initializing the dicts #
	device_dict = {}
	ip_dict = {}

	counter = 0                        # counter to manage the progress
	for row in reader:
		counter += 1

		# extracting ip address alone from the given input and store it in a list #
		ip_all_str = ','.join(row[2:]) 
		ip_list = []
		ip_all_list = ip_all_str.replace("{","").replace("}","").replace("),("," ").replace("(","").replace(")","").split(" ")     # formatting 
		for val in ip_all_list:
			ip_list.append(val.split(",")[0])

		# if device, write to device dict, else write to ip dict #
		if row[1] == '0':
			device_dict[row[0]] = ip_list
		elif row[1] == '1':
			for ip in ip_list:
				temp_list = ip_dict.get(ip,[])
				temp_list.append(row[0])
				ip_dict[ip] = temp_list
		else:
			print "Device or Cookie has unacceptable value.. Value : ", row[1]
			raise

		# printing the progress #
		if counter % 50000 == 0:
			print "Processed : ", counter
			
	return device_dict, ip_dict

In [ ]:
# Notes...either alter the path to the data file or just do the lazy way of placing this notebook in the same folder
# with the data

# Load files that lend themselves to a dataframe...(un)commment the data frames as you need them.

# Note..I think we could create sets and then do intersections on them to quickly collapse data frames...it's a ToDo
# on my list.

df_cookie_all_basic = pd.read_csv('cookie_all_basic.csv')
#df_dev_test_basic = pd.read_csv('dev_test_basic.csv')
#df_dev_train_basic = pd.read_csv('dev_train_basic.csv')
#df_ipagg_all = pd.read_csv('ipagg_all.csv')
#df_sample_submission = pd.read_csv('sampleSubmission.csv')

In [ ]:
df_cookie_all_basic.head()

In [ ]:
# file config #
data_path = "../Data/"
ip_file = open("id_all_ip.csv")

print "Getting device and IP dict.."
device_dict, ip_dict = getIPDict(ip_file)

df_device_dict = pd.DataFrame(list(device_dict.iteritems()), columns=['device_dict_key', 'device_dict_values'])

df_ip_dict = pd.DataFrame(list(ip_dict.iteritems()), columns=['ip_dict_key', 'ip_dict_values'])


In [ ]:
# This cell helps spit out the first 3 rows of the data frames for df_devices
df_device_dict.head(3)
df_ip_dict.head(3)

In [ ]:
# Call this if you need to run a separate console to help debug or try functions
%qtconsole

In [3]:
# Define/Declare functions here
def getIPDict_2(ip_file):
    """
    Fill in when done
    """
    # reading the ip file #
    reader = csv.reader(ip_file)
    header = reader.next()             # skipping the header

    # initializing the dicts #

    property_dict = {}

    counter = 0

    for row in reader:
        counter += 1
        #print(row[0])

        id_all = row[0]
        indicator = row[1]

        prop_all_str = ','.join(row[2:])

        prop_list = []
        count_list = []

        prop_all_list = prop_all_str.replace("{","").replace("}","").replace("),("," ").replace("(","").replace(")","").split(" ")
        #print(prop_all_list)

        for val in prop_all_list:
            prop_list.append(val.split(",")[0])
            count_list.append(val.split(",")[1])

        #print(prop_list)
        #print(count_list)
        
        property_dict[id_all] = (indicator, prop_list, count_list)

        # printing the progress #
        if counter % 50000 == 0:
            print "Processed : ", counter

    return property_dict

In [4]:
#ip_file = open("test.csv")
ip_file = open("id_all_property.csv")
test = getIPDict_2(ip_file)


df_property_dict = pd.DataFrame(list(test.iteritems()), columns=['id_dict_key', 'prop_dict_values'])

Processed :  50000
Processed :  100000
Processed :  150000
Processed :  200000
Processed :  250000
Processed :  300000
Processed :  350000
Processed :  400000
Processed :  450000
Processed :  500000
Processed :  550000
Processed :  600000
Processed :  650000
Processed :  700000
Processed :  750000
Processed :  800000
Processed :  850000
Processed :  900000
Processed :  950000
Processed :  1000000
Processed :  1050000
Processed :  1100000
Processed :  1150000
Processed :  1200000
Processed :  1250000
Processed :  1300000
Processed :  1350000
Processed :  1400000
Processed :  1450000
Processed :  1500000
Processed :  1550000
Processed :  1600000
Processed :  1650000
Processed :  1700000
Processed :  1750000
Processed :  1800000
Processed :  1850000
Processed :  1900000
Processed :  1950000
Processed :  2000000
Processed :  2050000
Processed :  2100000
Processed :  2150000


In [41]:
# This data frame uses an iterator to go through each row within the data frame. It applies several things:
# It creates a new column called 'total_sum' by using the apply method via lambda function.
# It first takes the nested tuple, takes out the column with property count, changes it from a string to int
# then it adds all the values togehter.


df_property_dict['total_sum'] = df_property_dict.prop_dict_values.apply(lambda row: sum(map(int, row[2])))

In [44]:
df_property_dict.head(3)

,id_dict_key,prop_dict_values,total_sum
0,id_1684982,"(0, [property_55876, property_490148, property...",23
1,id_4227995,"(1, [property_264003, property_210052, propert...",197
2,id_4651263,"(1, [property_168296], [1])",1


In [ ]:
# Want to sort the data frame for id with highest property sum
sort_id = df_property_dict(['total_sum'], ascending = [0])

# This will return a data frame with all the id by total sum descending....modify the data frames as needed for other opterations

